In [ ]:
import pandas as pd
import os

from vocab import Vocab
from constants import *
NDC_ATC_MAPPING = "ndc_map.csv"


# <span style="text-decoration: underline">Statistics</span>
This is the file where statistics are calculated

#### <u>Stat-1: Method to calculate % of visits that have higher than certain threshold % of "[unk]" tokens </u>

In [ ]:
def percentage_visits_with_high_unk(vocab: Vocab, threshold: float = 0.3, for_single_vsit: bool = False):
    visit_record_key = ["SUBJECT_ID", "HADM_ID", "ICD9_CODE", "ATC4"]
    visit = pd.read_pickle(os.path.join(GLOBAL_DATA_PATH, MULTI_VISIT_PKL))[visit_record_key[1:]]
    if for_single_vsit is True:
        visit = pd.read_pickle(os.path.join(GLOBAL_DATA_PATH, SINGLE_VISIT_PKL))[visit_record_key[1:]]
    total = visit.shape[0]
    def process(row):
        icd: list[str] = row["ICD9_CODE"]
        unk = 0
        for code in icd:
            if vocab.word2idx.get(code) == None:
                unk += 1
        if unk/len(icd) > threshold:
            print(row["HADM_ID"])
            return 1
    temp = visit.apply(process, axis=1)
    count: int = temp.loc[temp == 1].shape[0]
    return count/total

#### <u>Stat-2: Method to calculate stats of Table 2 from Original paper </u>

In [45]:
def table_2_stats():
    visit_record_key = ["SUBJECT_ID", "HADM_ID", "ICD9_CODE", "ATC4"]
    single_visit = pd.read_pickle(os.path.join(GLOBAL_DATA_PATH, SINGLE_VISIT_PKL))
    multi_visit_temporal = pd.read_pickle(os.path.join(GLOBAL_DATA_PATH, MULTI_VISIT_TEMPORAL_PKL))
    multi_visit = pd.read_pickle(os.path.join(GLOBAL_DATA_PATH, MULTI_VISIT_PKL))

    # Single visit stats
    single_visit_sbj_count = single_visit[visit_record_key[0]].nunique()
    print("[table_2_stats] # of patients (Single-Visit)", single_visit_sbj_count)
    total_visits = single_visit[visit_record_key[:2]].groupby("SUBJECT_ID").count().sum().values[0]
    print("[table_2_stats] avg. # of visits (Single-Visit)", total_visits/single_visit_sbj_count)
    total_dx = single_visit[visit_record_key[2]].apply(len).sum()
    print("[table_2_stats] avg. # of dx per visit per patient (Single-Visit)", total_dx/total_visits)
    total_rx = single_visit[visit_record_key[3]].apply(len).sum()
    print("[table_2_stats] avg. # of rx per visit per patient (Single-Visit)", total_rx/total_visits)
    unique_dx = len(set(code for ls in single_visit[visit_record_key[2]] for code in ls))
    print("[table_2_stats] unique # of dx (Single-Visit)", unique_dx) # 7k since we do not filter patient data
    # we replace in training/eval less frequent DX with "[UNK]" && similar for RX
    unique_rx = len(set(code for ls in single_visit[visit_record_key[3]] for code in ls))
    print("[table_2_stats] unique # of drx (Single-Visit)", unique_rx)

    # Multiple visit stats
    multi_visit_sbj_count = multi_visit_temporal[visit_record_key[0]].nunique()
    print("\n\n[table_2_stats] # of patients (Multi-Visit)", multi_visit_sbj_count)
    total_multi_visits = (multi_visit_temporal.groupby(by=["SUBJECT_ID"])["T_1"].max() + 1).sum()
    print("[table_2_stats] avg. # of visits (Multi-Visit)", total_multi_visits/multi_visit_sbj_count)
    def process_icd(row: pd.Series):
        row["ICD9_LEN"] = sum([len(ls) for ls in row["ICD9_CODE"]])
        return row
    # this can be improved but not P1    
    total_multi_dx = multi_visit_temporal.apply(process_icd, axis=1).groupby("SUBJECT_ID")["ICD9_LEN"].max().sum() 
    print("[table_2_stats] avg. # of dx per visit per patient (Multi-Visit)(This includes the first visit dx as well)"\
        , total_multi_dx/total_multi_visits) # though it is fall as seen in paper but it is because we are including both 
    def process_atc(row: pd.Series):
        row["ATC4_LEN"] = sum([len(ls) for ls in row["ATC4"]])
        return row
    # this can be improved but not P1    
    total_multi_rx = multi_visit_temporal.apply(process_atc, axis=1).groupby("SUBJECT_ID")["ATC4_LEN"].max().sum() 
    print("[table_2_stats] avg. # of rx per visit per patient (Multi-Visit)(This includes the first visit rx as well)"\
        , total_multi_rx/total_multi_visits) # though it is fall as seen in paper but it is because we are including both 
    unique_dx_multi = set()
    def store_unique_icd(row: pd.Series):
        unique_dx_multi.update(code for ls in row["ICD9_CODE"] for code in ls)
        return row
    multi_visit_temporal.apply(store_unique_icd, axis=1) 
    print("[table_2_stats] unique # of dx (Multi-Visit)", len(unique_dx_multi))
    unique_rx_multi = set()
    def store_unique_atc(row: pd.Series):
        unique_rx_multi.update(code for ls in row["ATC4"] for code in ls)
        return row
    multi_visit_temporal.apply(store_unique_atc, axis=1) 
    print("[table_2_stats] unique # of rx (Multi-Visit)", len(unique_rx_multi))


[table_2_stats] # of patients (Single-Visit) 29189
[table_2_stats] avg. # of visits (Single-Visit) 1.0
[table_2_stats] avg. # of dx per visit per patient (Single-Visit) 11.444276953646922
[table_2_stats] avg. # of rx per visit per patient (Single-Visit) 21.20703004556511
[table_2_stats] unique # of dx (Single-Visit) 6191
[table_2_stats] unique # of drx (Single-Visit) 398


[table_2_stats] # of patients (Multi-Visit) 5917
[table_2_stats] avg. # of visits (Multi-Visit) 2.677539293560926
[table_2_stats] avg. # of dx per visit per patient (Multi-Visit)(This includes the first visit dx as well) 14.217572429464116
[table_2_stats] avg. # of rx per visit per patient (Multi-Visit)(This includes the first visit rx as well) 22.555071640472132
[table_2_stats] unique # of dx (Multi-Visit) 4551
[table_2_stats] unique # of rx (Multi-Visit) 385


#### <u>Stat-3: Method to calculate stats of Table 2 from Original paper using csv from MIMIC-III</u>

In [ ]:
def preprocess_mapping_file():
    ndc_mapping = pd.read_csv(os.path.join(GLOBAL_DATA_PATH, NDC_ATC_R_SCRIPT_MAPPING)).dropna()\
        .drop_duplicates()
    def process_row(row: pd.Series):
        row["NDC"]= float(str(row["NDC"]).replace("-", ""))
        return row
    ndc_mapping = ndc_mapping.apply(process_row, axis=1).dropna().drop_duplicates()
    ndc_mapping.to_csv(os.path.join(GLOBAL_DATA_PATH, NDC_ATC_MAPPING))

In [44]:
def table_2_stats_using_csv():
    visit_record_key = ["SUBJECT_ID", "HADM_ID", "ICD9_CODE", "ATC4"]
    admissions = pd.read_csv(os.path.join(GLOBAL_DATA_PATH, ADMISSIONS))[visit_record_key[:2] + ["ADMITTIME"]]\
        .dropna().drop_duplicates()
    diagnosis_icd = pd.read_csv(os.path.join(GLOBAL_DATA_PATH, DIAGNOSIS_ICD))[visit_record_key[:3]].dropna()\
        .drop_duplicates()
    prescriptions = pd.read_csv(os.path.join(GLOBAL_DATA_PATH, PRESCRIPTIONS))[["SUBJECT_ID", "HADM_ID", "NDC"]]
    ndc_mapping = pd.read_csv(os.path.join(GLOBAL_DATA_PATH, NDC_ATC_MAPPING)).dropna()\
        .drop_duplicates()
        
    # Single visit stats
    # Intersecting with prescriptions table since some patients have admission but no medications
    # we are not concerned about them for this experiment
    intersection = admissions.loc[admissions["SUBJECT_ID"].isin(prescriptions["SUBJECT_ID"])]
    work_table_1 = intersection.groupby([visit_record_key[0]])[visit_record_key[1]].count() #SUBJECT_ID, HADM_ID
    single_visit_sbj = work_table_1.loc[work_table_1 == 1].reset_index()[visit_record_key[0]] #SUBJECT_ID
    print("[table_2_stats] # of patients (Single-Visit)", single_visit_sbj.shape[0])
    total_visits = work_table_1[single_visit_sbj].sum()
    print("[table_2_stats] avg. # of visits (Single-Visit)", total_visits/single_visit_sbj.shape[0])
    work_table_2 = pd.merge(single_visit_sbj, diagnosis_icd, on=["SUBJECT_ID"]).dropna().drop_duplicates() #SUBJECT_ID, HADM_ID, ICD9_CODE
    total_dx = work_table_2["ICD9_CODE"].shape[0]
    print("[table_2_stats] avg. # of dx per visit per patient (Single-Visit)", total_dx/total_visits)
    work_table_3 = pd.merge(prescriptions, ndc_mapping, on=["NDC"])[visit_record_key[:2] + ["ATC4"]] # SUBJECT_ID, HADM_ID
    work_table_3 = work_table_3.dropna().drop_duplicates() # since repeated ATC4 codes for same SUBJECT_ID and HADM_ID
    # ATC4
    work_table_4 = pd.merge(work_table_3, single_visit_sbj, on=["SUBJECT_ID"])
    total_rx = work_table_4.shape[0]
    print("[table_2_stats] avg. # of rx per visit per patient (Single-Visit)", total_rx/total_visits)
    unique_dx = work_table_2["ICD9_CODE"].dropna().nunique()
    print("[table_2_stats] unique # of dx (Single-Visit)", unique_dx) # 7k since we do not filter patient data
    unique_rx = work_table_4["ATC4"].dropna().nunique()
    print("[table_2_stats] unique # of drx (Single-Visit)", unique_rx)

    # # Multiple visit stats
    multi_visit_sbj = work_table_1.loc[work_table_1 > 1].reset_index()[visit_record_key[0]] #SUBJECT_ID
    print("\n\n[table_2_stats] # of patients (Multi-Visit)", multi_visit_sbj.shape[0])
    total_multi_visits = work_table_1.loc[work_table_1 > 1].sum()
    print("[table_2_stats] avg. # of visits (Multi-Visit)", total_multi_visits/multi_visit_sbj.shape[0])
    work_table_5 = pd.merge(multi_visit_sbj, diagnosis_icd, on=["SUBJECT_ID"]).dropna().drop_duplicates() #SUBJECT_ID, HADM_ID, ICD9_CODE
    print("[table_2_stats] avg. # of dx per visit per patient (Multi-Visit)(This includes the first visit dx as well)"\
        , work_table_5.shape[0]/total_multi_visits) # it increases because we are including both  
    work_table_6 = pd.merge(work_table_3, multi_visit_sbj, on=["SUBJECT_ID"])["ATC4"]
    total_multi_rx = work_table_6.shape[0]
    print("[table_2_stats] avg. # of rx per visit per patient (Multi-Visit)(This includes the first visit rx as well)"\
        , total_multi_rx/total_multi_visits) # it increases because we are including both 
    print("[table_2_stats] unique # of dx (Multi-Visit)", work_table_5["ICD9_CODE"].nunique())
    print("[table_2_stats] unique # of rx (Multi-Visit)", work_table_6.nunique())

[table_2_stats] # of patients (Single-Visit) 32212
[table_2_stats] avg. # of visits (Single-Visit) 1.0
[table_2_stats] avg. # of dx per visit per patient (Single-Visit) 10.985409164286601
[table_2_stats] avg. # of rx per visit per patient (Single-Visit) 18.816621134980753
[table_2_stats] unique # of dx (Single-Visit) 6278
[table_2_stats] unique # of drx (Single-Visit) 294


[table_2_stats] # of patients (Multi-Visit) 7151
[table_2_stats] avg. # of visits (Multi-Visit) 2.6785065025870507
[table_2_stats] avg. # of dx per visit per patient (Multi-Visit)(This includes the first visit dx as well) 13.318314712331627
[table_2_stats] avg. # of rx per visit per patient (Multi-Visit)(This includes the first visit rx as well) 19.283543907277853
[table_2_stats] unique # of dx (Multi-Visit) 4852
[table_2_stats] unique # of rx (Multi-Visit) 292
